## Creating sub-agent 2: Get technical analysis of the financial data

In this folder we will create the second sub-agent for collecting technical analysis to user questions. This agent will call the financial data API to handle the following user questions:

1. Questions about getting stock prices. Based on a user question, ticker over a given data range and interval, the agent will get the stock prices.

1. Questions about seeking information on the current stock price. If the user asks for the current stock price, then this agent will call a lambda function to get the current (latest) stock price for a ticker

1. If a user question is about calculating technical indicators for a given ticker and a time period, for example `RSI`, `MACD`, `SMA`, `EMA` and `Bollinger Bands` calculations.

### Prerequisites
---

You are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

Before running this notebook, make sure to populate the `.env` file with your `FINANCIAL_DATASETS_API_KEY` and `TAVILY_API_KEY`. These two `API` keys will be used by the agent to access information on user related questions. You can create an `API` here: https://www.financialdatasets.ai/

In [1]:
!pip uninstall boto3 botocore awscli --yes

Found existing installation: boto3 1.35.83
Uninstalling boto3-1.35.83:
  Successfully uninstalled boto3-1.35.83
Found existing installation: botocore 1.35.83
Uninstalling botocore-1.35.83:
  Successfully uninstalled botocore-1.35.83
Found existing installation: awscli 1.36.24
Uninstalling awscli-1.36.24:
  Successfully uninstalled awscli-1.36.24


In [2]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

### Import required libraries

Next we will import the required libraries. We will also import some support functions available in the parent directory. Those functions are:

1. `create_agent`: helps you to create the necessary IAM permissions and Bedrock agetns based on the agent's name, instructions, foundation models, descriptions and other properties.

1. `invoke_agent_helper`: helps you to invoke your agent using invoke_agent
You can see the implementation of both functions in the parent directory

In [3]:
import os
import sys
import time
import json
import boto3
from dotenv import load_dotenv

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from utils import *
from globals import *
from agents import create_agent, invoke_agent_helper

/Users/madhurpt/Desktop/hedge-fund-analyst-multi-agent-collaboration-2
Boto3 version: 1.35.83


[2024-12-17 19:41:06,745] p87293 {requests.py:227} INFO - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
[2024-12-17 19:41:06,836] p87293 {requests.py:263} INFO - <<< {"data":{"viewer":{"username":"madhur-prash","defaultEntity":{"name":"madhur-prash-none"}}}}
[2024-12-17 19:41:08,200] p87293 {utils.py:160} INFO - NumExpr defaulting to 8 threads.
[2024-12-17 19:41:08,975] p87293 {_client.py:1026} INFO - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
[2024-12-17 19:41:09,511] p87293 {config.py:59} INFO - PyTorch version 2.3.0 available.
[2024-12-17 19:41:09,695] p87293 {requests.py:227} INFO - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
[2024-12-17 19:41:09,781] p87293 {requests.py:263} INFO - <<< {"data":{"viewer":{"username":"madhur-prash","defaultEntity":{"name":"madhur-pr

Logged in as Weights & Biases user: madhur-prash.
View Weave data at https://wandb.ai/madhur-prash-none/multi-agent-collaboration/weave


In [15]:
# Load environment variables from .env file
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [5]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Defining agent configuration
---

In [6]:
agent_instruction = """You are a technical analysis assistant that helps users analyze stock price movements and technical indicators. You can calculate and interpret various technical indicators using historical price data.

You have access to the following technical analysis capabilities based on the API key you have access to to call the available functions and get the following information:

1. Stock Price Data:
   - Current stock prices
   - Historical price data with various intervals
   - Custom date range analysis

2. Technical Indicators:
   - RSI
   - MACD
   - SMA
   - EMA
   - Bollinger Bands

You require the user to provide:
1. A stock ticker symbol
2. The type of technical indicator they want to analyze
3. Optionally: specific time periods or date ranges

Available functions:
1. get_current_stock_price: Retrieve latest stock price
2. get_stock_prices: Get historical price data for a specified period
3. get_technical_indicators: Calculate technical indicators for analysis

If you do not have access to the data that the user is asking for, do not make up an answer. Be completely accurate and only provide analysis based on the available technical indicators and price data.

Only answer questions related to technical analysis and price data based on the provided functions. If unsure, acknowledge limitations"""

agent_description = "Agent for technical analysis of stocks using various technical indicators and price data"

In [7]:
# This is the function definition for the lambda function that will be invoked
# as a part of the action group
functions = [{
    'name': 'get_stock_prices',
    'description': 'Get prices for a ticker over a given date range and interval.',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "start_date": {
            "description": "Start date to get the stock price from",
            "required": True,
            "type": "string"
        },
        "end_date": {
            "description": "End data until which the stock price needs to be computed",
            "required": True,
            "type": "string"
        }, 
        "limit": {
            "description": "number of statements to retrieve",
            "required": True,
            "type": "integer"
        }
    }
},
{
    'name': 'get_current_stock_price',
    'description': 'Get the current (latest) stock price for a ticker.',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        }
    }
},
{
    'name': 'get_technical_indicators',
    'description': 'Calculate technical indicators (RSI, MACD, SMA, EMA, or Bollinger Bands) for a given ticker.',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "indicator": {
            "description": "technical indicator type (RSI, MACD, SMA, EMA, or BBANDS)",
            "required": True,
            "type": "string"
        },
        "period": {
            "description": "period for indicator calculation (default: 14)",
            "required": False,
            "type": "integer"
        },
        "start_date": {
            "description": "start date for analysis (YYYY-MM-DD)",
            "required": False,
            "type": "string"
        },
        "end_date": {
            "description": "end date for analysis (YYYY-MM-DD)",
            "required": False,
            "type": "string"
        }
    }
}]

### Add a lambda layer to the lambda function
---

In this section of the notebook, we will add a lambda layer to the lambda function. We will be installing `requests`, `ta` to make requests to the financial data API key.

In [8]:
# Create and publish the layer ~ this step will take around 2 minutes

# In this case we want to add a layer to the lambda containing files to import the 
# requests, ta and pandas libraries
layer_zip = create_lambda_layer(['requests'])
layer_arn = publish_layer('technical-agent-lambda-layer-new')

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
langchain-community 0.2.17 requires langchain-core<0.3.0,>=0.2.39, but you have langchain-core 0.3.24 which is incompatible.
langchain-community 0.2.17 requires langsmith<0.2.0,>=0.1.112, but you have langsmith 0.2.3 which is incompatible.
langchain-community 0.2.17 requires tenacity!=8.4.0,<9.0.0,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.3 which is incompatible.
streamlit 1.30.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
cohere 5.3.4 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.20.1 which is incompatible.
llam

### Create agent
---

Next we will create the agent with the provided information. In this, we will use the prompt configuration, the lambda function and lambda layers, update the lambda function to contain the `FINANCIAL_DATASETS_API_KEY` that the agent can use to fetch data to user inputs, and then finally create the agent.

In [9]:
action_group_config = {
    'name': 'TechnicalAgentActionGroup',
    'description': 'Action group for technical analysis of stocks using various technical indicators and price data',
    'functions': functions,
    'lambda_function_name': f'{SUB_AGENT_NAME_TECHNICAL_ANALYST}-lambda',
    'lambda_file_path': TECHNICAL_LAMBDA_FUNCTION_NAME,
    'dynamodb_table_name': f'{SUB_AGENT_NAME_TECHNICAL_ANALYST}-table',
    'dynamodb_attribute_name': 'analysis-id', 
    'lambda_layers': [layer_arn]
}

# Create a Lambda client and attach the API key as env variable to the lambda function
lambda_client = boto3.client('lambda')
lambda_function_name = action_group_config['lambda_function_name']
environment_variables = {
    'FINANCIAL_DATASETS_API_KEY': os.getenv('FINANCIAL_DATASETS_API_KEY')
}

In [10]:
# Update your agent creation code to include the action_group_config
technical_analyst_agent_id, technical_analyst_agent_alias_id, technical_analyst_agent_alias_arn = create_agent(
    SUB_AGENT_NAME_TECHNICAL_ANALYST,
    agent_instruction,
    agent_foundation_model=NOVA_LITE,
    agent_description=agent_description,
    action_group_config=action_group_config
)

creating agent
Policy technical-analyst-agent-ba already exists
Checking if AmazonBedrockExecutionRoleForAgents_technical-analyst-agent role also exists
Detaching and deleting technical-analyst-agent-ba
deleting AmazonBedrockExecutionRoleForAgents_technical-analyst-agent
Recreating technical-analyst-agent-ba
creating and attaching action group
Role already exists -- deleting and creating it again
Detaching AWSLambdaBasicExecutionRole
Detaching AmazonDynamoDBFullAccess
deleting technical-analyst-agent-lambda-role
recreating technical-analyst-agent-lambda-role
attaching basic lambda permissions to technical-analyst-agent-lambda-role
attaching dynamodb permissions to technical-analyst-agent-lambda-role
technical-analyst-agent-lambda already exists, deleting it and recreating
Agent id BRU2RYCZR1 current status: NOT_PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id BRU2RYCZR1 current status: PREPARED
Waiting for agent status to change. Current status VERSIONING


In [11]:
# Update the Lambda function's configuration to include the environment variables
# In the case of this agent, we need the financial dataset API as an env variable passed to the lambda
# so that it can be used in fetching data based on user questions
response = lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Environment={
        'Variables': environment_variables
    },
    Layers=action_group_config['lambda_layers']
)

### Getting details from the agent
--- 
Let's take a look at the details from the created agent.

In [12]:
technical_analyst_agent_id, technical_analyst_agent_alias_id, technical_analyst_agent_alias_arn

('BRU2RYCZR1',
 'X0OOOOEXXL',
 'arn:aws:bedrock:us-east-1:218208277580:agent-alias/BRU2RYCZR1/X0OOOOEXXL')

### Testing the Fundamental Analyst agent
---

Now that we've created the agent, let's test it by using our `invoke_agent_helper` function. Here, we will invoke the `technical analyst` agent to provide information on `technical indicators` and `stock price data` based on the question provided by the user.

Since the `invoke_agent_helper` is wrapped with a `weave` decorator, it will log the input/outputs or any errors to your weave account dashboard. To create a weave API key, refer to the following link: https://wandb.ai/site/weave/

In [13]:
# Sleep for 30 seconds before invoking the technical analyst agent
time.sleep(30)

In [16]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """What is the latest stock price for AAPL from january 2022 to december 2022?"""
response = invoke_agent_helper(
    query, session_id, technical_analyst_agent_id, technical_analyst_agent_alias_id, enable_trace=True
)
print(response)

[2024-12-17 19:43:10,208] p87293 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:43:10,208] p87293 {agents.py:761} INFO - {
  "RequestId": "3fd3b3c1-2cda-4038-b1e2-d5bf1a2f3eea",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:43:10 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "3fd3b3c1-2cda-4038-b1e2-d5bf1a2f3eea",
    "x-amz-bedrock-agent-session-id": "0dff3c0c-bcd9-11ef-8c47-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:43:10,365] p87293 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:43:10,366] p87293 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/BRU2RYCZR1/X0OOOOEXXL"
  }
]
[2024-12-17 19:43:10,366] p87293 {agents.py:803} INFO - Orchestration Trace Detected
[2024

Exception: ('unexpected event.', EventStreamError('An error occurred (dependencyFailedException) when calling the InvokeAgent operation: The server encountered an error processing the Lambda response. Check the Lambda response and retry the request'))

In [17]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """What is the latest SMA for AAPL in the first 20 days of the year 2021?"""
response = invoke_agent_helper(
    query, session_id, technical_analyst_agent_id, technical_analyst_agent_alias_id, enable_trace=True
)
print(response)

[2024-12-17 19:43:26,740] p87293 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:43:26,741] p87293 {agents.py:761} INFO - {
  "RequestId": "bf0ad130-727a-4d5b-968e-d67231fd383b",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:43:26 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "bf0ad130-727a-4d5b-968e-d67231fd383b",
    "x-amz-bedrock-agent-session-id": "17dfdc40-bcd9-11ef-8c47-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:43:26,904] p87293 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:43:26,905] p87293 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/BRU2RYCZR1/X0OOOOEXXL"
  }
]
[2024-12-17 19:43:26,905] p87293 {agents.py:803} INFO - Orchestration Trace Detected
[2024

Exception: ('unexpected event.', EventStreamError('An error occurred (dependencyFailedException) when calling the InvokeAgent operation: The server encountered an error processing the Lambda response. Check the Lambda response and retry the request'))

In [18]:
%store technical_analyst_agent_id
%store technical_analyst_agent_alias_id
%store technical_analyst_agent_alias_arn

Stored 'technical_analyst_agent_id' (str)
Stored 'technical_analyst_agent_alias_id' (str)
Stored 'technical_analyst_agent_alias_arn' (str)
